In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [32]:
def load_data(transactions_csv, products_csv, shops_csv):
    """Load data from CSV files and combine them"""
    print("Loading CSV files...")
    
    # Load the three tables
    transactions = pd.read_csv(transactions_csv)
    products = pd.read_csv(products_csv)
    shops = pd.read_csv(shops_csv)
    
    print(f"Loaded {len(transactions)} transactions")
    print(f"Loaded {len(products)} products")
    print(f"Loaded {len(shops)} shops")
    
    # Combine all data
    data = transactions.merge(products, on='product_id', how='left')
    data = data.merge(shops, on='shop_id', how='left')
    
    # Filter only sales transactions
    if 'transaction_type' in data.columns:
        data = data[data['transaction_type'] == 'SALE']
    
    # Convert transaction_time to datetime
    data['transaction_time'] = pd.to_datetime(data['transaction_time'])
    data = data.sort_values('transaction_time')
    
    print(f"Combined data: {len(data)} sales records")
    print(f"Date range: {data['transaction_time'].min()} to {data['transaction_time'].max()}")
    
    return data


In [ ]:
def prepare_monthly_data(data):
    """Convert daily transactions to monthly sales data"""
    print("Preparing monthly sales data...")
    
    # Create year-month column
    data['year_month'] = data['transaction_time'].dt.to_period('M')
    
    # Group by product, shop, and month to get monthly totals
    monthly_sales = data.groupby(['product_id', 'shop_id', 'year_month']).agg({
        'quantity': 'sum',
        'total_amount': 'sum',
        'unit_price': 'mean',
        'product_name': 'first',
        'category': 'first',
        'city': 'first',
        'standard_price': 'first'
    }).reset_index()
    
    # Rename columns for clarity
    monthly_sales.columns = ['product_id', 'shop_id', 'year_month', 'monthly_quantity', 
                            'monthly_revenue', 'avg_price', 'product_name', 'category', 
                            'city', 'standard_price']
    
    monthly_sales = monthly_sales.sort_values(['product_id', 'shop_id', 'year_month'])
    
    print(f"Created {len(monthly_sales)} monthly sales records")
    return monthly_sales

def create_features(monthly_data):
    """Create features for prediction"""
    print("Creating features...")
    
    # Convert year_month to datetime
    monthly_data['month_date'] = monthly_data['year_month'].dt.to_timestamp()
    monthly_data['month'] = monthly_data['month_date'].dt.month
    monthly_data['year'] = monthly_data['month_date'].dt.year
    
    # Create lag features (previous months' sales)
    monthly_data['last_month_qty'] = monthly_data.groupby(['product_id', 'shop_id'])['monthly_quantity'].shift(1)
    monthly_data['last_2_months_qty'] = monthly_data.groupby(['product_id', 'shop_id'])['monthly_quantity'].shift(2)
    monthly_data['last_3_months_qty'] = monthly_data.groupby(['product_id', 'shop_id'])['monthly_quantity'].shift(3)
    
    # Create average of last 3 months
    monthly_data['avg_last_3_months'] = (
        monthly_data['last_month_qty'] + 
        monthly_data['last_2_months_qty'] + 
        monthly_data['last_3_months_qty']
    ) / 3
    
    # Create trend feature
    monthly_data['trend'] = monthly_data['last_month_qty'] - monthly_data['last_2_months_qty']
    
    # Price difference from standard
    monthly_data['price_difference'] = monthly_data['avg_price'] - monthly_data['standard_price']
    
    # Seasonal features
    monthly_data['is_holiday_month'] = monthly_data['month'].isin([11, 12, 1, 4, 10]).astype(int)
    monthly_data['is_summer'] = monthly_data['month'].isin([3, 4, 5, 6]).astype(int)
    
    # Convert categorical variables to numbers
    monthly_data['category_code'] = pd.Categorical(monthly_data['category']).codes
    monthly_data['city_code'] = pd.Categorical(monthly_data['city']).codes
    
    # Remove rows with missing values
    monthly_data = monthly_data.dropna(subset=['last_month_qty', 'last_2_months_qty', 'last_3_months_qty'])
    
    print(f"Final dataset: {len(monthly_data)} records with features")
    return monthly_data

In [ ]:
def train_model(monthly_data):
    """Train the prediction model"""
    print("Training model...")
    
    # Select features for training
    feature_columns = [
        'last_month_qty', 'last_2_months_qty', 'last_3_months_qty',
        'avg_last_3_months', 'trend', 'month', 'avg_price',
        'standard_price', 'price_difference', 'is_holiday_month',
        'is_summer', 'category_code', 'city_code'
    ]
    
    # Prepare training data
    X = monthly_data[feature_columns]
    y = monthly_data['monthly_quantity']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Test model
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Model trained successfully!")
    print(f"Mean Absolute Error: {mae:.2f}")
    print(f"Root Mean Square Error: {rmse:.2f}")
    print(f"Root Mean Square Error: {accuracy:.2f}")
    # Show feature importance
    importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nMost Important Features:")
    print(importance.head(6))
    
    return model, feature_columns, {'mae': mae, 'rmse': rmse, 'accuracy': accuracy, 'feature_importance': importance}

In [35]:
def predict_next_month(model, feature_columns, monthly_data, product_id, shop_id):
    """Predict next month's sales for a specific product and shop"""
    
    # Get data for the specific product-shop combination
    product_data = monthly_data[
        (monthly_data['product_id'] == product_id) & 
        (monthly_data['shop_id'] == shop_id)
    ].copy()
    
    if len(product_data) == 0:
        return None, "No historical data found for this product-shop combination"
    
    # Get the most recent record
    latest_record = product_data.sort_values('year_month').iloc[-1]
    
    # Create features for prediction
    features = {
        'last_month_qty': latest_record['monthly_quantity'],
        'last_2_months_qty': latest_record['last_month_qty'],
        'last_3_months_qty': latest_record['last_2_months_qty'],
        'avg_last_3_months': latest_record['avg_last_3_months'],
        'trend': latest_record['trend'],
        'month': (latest_record['month'] % 12) + 1,  # Next month
        'avg_price': latest_record['avg_price'],
        'standard_price': latest_record['standard_price'],
        'price_difference': latest_record['price_difference'],
        'is_holiday_month': ((latest_record['month'] % 12) + 1) in [11, 12, 1, 4, 10],
        'is_summer': ((latest_record['month'] % 12) + 1) in [3, 4, 5, 6],
        'category_code': latest_record['category_code'],
        'city_code': latest_record['city_code']
    }
    
    # Convert to DataFrame for prediction
    X_pred = pd.DataFrame([features])
    
    # Make prediction
    prediction = model.predict(X_pred)[0]
    prediction = max(0, prediction)  # Ensure non-negative
    
    return round(prediction, 2), "Success"


In [36]:
def get_all_predictions(model, feature_columns, monthly_data):
    """Get predictions for all product-shop combinations"""
    predictions = []
    
    # Get unique product-shop combinations
    combinations = monthly_data[['product_id', 'shop_id', 'product_name', 'city']].drop_duplicates()
    
    for _, row in combinations.iterrows():
        prediction, status = predict_next_month(model, feature_columns, monthly_data, 
                                              row['product_id'], row['shop_id'])
        
        predictions.append({
            'product_id': row['product_id'],
            'product_name': row['product_name'],
            'shop_id': row['shop_id'],
            'city': row['city'],
            'predicted_quantity': prediction,
            'status': status
        })
    
    return pd.DataFrame(predictions)

In [37]:
def predict_sales(transactions_csv, products_csv, shops_csv):
    """Main function to predict sales from CSV files"""
    
    # Load and prepare data
    data = load_data(transactions_csv, products_csv, shops_csv)
    monthly_data = prepare_monthly_data(data)
    monthly_data = create_features(monthly_data)
    
    # Train model
    model, feature_columns, metrics = train_model(monthly_data)
    
    # Get all predictions
    predictions = get_all_predictions(model, feature_columns, monthly_data)
    
    return predictions, metrics

In [38]:
def predict_for_specific_product(transactions_csv, products_csv, shops_csv, product_id, shop_id):
    """Predict for a specific product and shop"""
    
    # Load and prepare data
    data = load_data(transactions_csv, products_csv, shops_csv)
    monthly_data = prepare_monthly_data(data)
    monthly_data = create_features(monthly_data)
    
    # Train model
    model, feature_columns, _ = train_model(monthly_data)
    
    # Make prediction
    prediction, status = predict_next_month(model, feature_columns, monthly_data, product_id, shop_id)
    
    return prediction, status

In [43]:
def view_predictions_file(filename=None):
    """View the most recent predictions file"""
    import glob
    
    if filename is None:
        # Find the most recent predictions file
        files = glob.glob("sales_predictions_*.csv")
        if not files:
            print("No predictions files found!")
            return
        filename = max(files)  # Most recent file
    
    try:
        predictions = pd.read_csv(filename)
        print(f"\n=== VIEWING PREDICTIONS FROM {filename} ===")
        display_predictions_summary(predictions)
        return predictions
    except FileNotFoundError:
        print(f"File {filename} not found!")
        return None
    """Generate sample CSV files for testing"""
    import random
    from datetime import datetime, timedelta
    
    print("Creating sample transactions.csv...")
    # Sample transactions
    transactions = []
    start_date = datetime(2023, 1, 1)
    
    for i in range(1000):
        date = start_date + timedelta(days=random.randint(0, 400))
        transactions.append({
            'transaction_id': i + 1,
            'product_id': random.randint(1, 20),
            'shop_id': random.randint(1, 5),
            'transaction_time': date.strftime('%Y-%m-%d %H:%M:%S'),
            'quantity': random.randint(1, 10),
            'unit_price': round(random.uniform(10, 100), 2),
            'total_amount': 0,  # Will calculate
            'transaction_type': 'SALE'
        })
    
    # Calculate total_amount
    for t in transactions:
        t['total_amount'] = t['quantity'] * t['unit_price']
    
    pd.DataFrame(transactions).to_csv('transactions.csv', index=False)
    
    print("Creating sample products.csv...")
    # Sample products
    categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Home']
    products = []
    for i in range(1, 21):
        products.append({
            'product_id': i,
            'product_name': f'Product {i}',
            'category': random.choice(categories),
            'standard_price': round(random.uniform(15, 90), 2)
        })
    
    pd.DataFrame(products).to_csv('products.csv', index=False)
    
    print("Creating sample shops.csv...")
    # Sample shops
    cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']
    shops = []
    for i in range(1, 6):
        shops.append({
            'shop_id': i,
            'shop_name': f'Shop {i}',
            'city': cities[i-1],
            'state': 'State'
        })
    
    pd.DataFrame(shops).to_csv('shops.csv', index=False)
    print("Sample data files created successfully!")

In [49]:
if __name__ == "__main__":
    
    # Check if CSV files exist locally first
    import os
    csv_files = ["transactions.csv", "products.csv", "shops.csv"]
    files_exist = all(os.path.exists(file) for file in csv_files)
    
    if files_exist:
        try:
            print("=== PREDICTING SALES FOR ALL PRODUCTS ===")
            predictions, metrics = predict_sales("transactions.csv", "products.csv", "shops.csv")
            
            # Display detailed summary
            display_predictions_summary(predictions)

            save_predictions_to_file(predictions, metrics)
            
            
            
            print("\nTop 10 Predictions:")
            top_predictions = predictions.head(10)
            for _, row in top_predictions.iterrows():
                print(f"Product: {row['product_name']} | Shop: {row['shop_id']} | "
                      f"City: {row['city']} | Predicted: {row['predicted_quantity']} units")
            
            print(f"\n=== USAGE EXAMPLES ===")
            print("To view saved predictions file:")
            print("predictions_df = view_predictions_file()")
            print("\nTo make predictions without saving:")
            print("predictions, metrics = predict_sales('transactions.csv', 'products.csv', 'shops.csv', save_to_file=False)")
            print("\nTo predict for specific product:")
            print("prediction, status = predict_for_specific_product('transactions.csv', 'products.csv', 'shops.csv', product_id=123, shop_id=45)")
        
        except Exception as e:
            print(f"Error processing files: {e}")
    
    else:
        # Generate sample data if files don't exist
        print("CSV files not found. Generating sample data...")
        generate_sample_data()
        
        print("Sample data created! Now running predictions...")
        try:
            predictions, metrics = predict_sales("transactions.csv", "products.csv", "shops.csv")
            
            print("\nTop 10 Predictions:")
            top_predictions = predictions.head(10)
            for _, row in top_predictions.iterrows():
                print(f"Product: {row['product_name']} | Shop: {row['shop_id']} | "
                      f"City: {row['city']} | Predicted: {row['predicted_quantity']} units")
        except Exception as e:
            print(f"Error: {e}")

=== PREDICTING SALES FOR ALL PRODUCTS ===
Loading CSV files...
Loaded 10000 transactions
Loaded 200 products
Loaded 50 shops
Combined data: 10000 sales records
Date range: 2023-01-01 00:00:00 to 2023-12-31 00:00:00
Preparing monthly sales data...
Created 9625 monthly sales records
Creating features...
Final dataset: 141 records with features
Training model...
Model trained successfully!
Mean Absolute Error: 2.59
Root Mean Square Error: 3.10

Most Important Features:
              feature  importance
8    price_difference    0.154062
4               trend    0.111852
11      category_code    0.106201
3   avg_last_3_months    0.102361
6           avg_price    0.085630
2   last_3_months_qty    0.081210

=== DETAILED PREDICTIONS SUMMARY ===

TOP 20 HIGHEST PREDICTED SALES:
 1. Lays Sna60           | Shop: 39 | City: Bhaktapur    | Predicted:   9.01 units
 2. Lux Per36            | Shop: 41 | City: Bhaktapur    | Predicted:   8.97 units
 3. Wai Wai Sna13        | Shop: 14 | City: Lalitpur  

No predictions files found!
